In [4]:
import xlrd
import math as m 


class nodo(): 
	def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia): 
		self.ide=ide
		self.lat=lat
		self.lon=lon
		self.nome=nome
		self.stato=stato
		self.identificativo=identificativo
		self.defi=defi
		self.pot=pot
		self.coll=coll
		self.adia=adia

# ide: città che mancano. (acora...) è il i del for.  
#defi: per rendere definitivo il nodo - citta.  true: se da controllare. quando passa, cambiano a false
#potenziale: peso: 2,4,6,8.
#collegamento: citta prima, per stampare al contrario, dalla fine torna a londra. 
#adia: adiacente: 3  nodi piu vicini, la lista che contiene il dizionario con le 3 piu vicine e i suoi pesi.

def dist2punti(latA,lonA,latB,lonB):
	Rt=6378.388  #radio terreste, per calcolare km
	lat_A = (m.pi*latA)/180.  #per calcolare arcocoseno,
	lat_B = (m.pi*latB)/180.
	lon_A = (m.pi*lonA)/180.
	lon_B = (m.pi*lonB)/180.
	dl=abs(lon_A-lon_B) #differenza in valore assoluto

	p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl))) #formula per calcolare distanza su sup sferica
	if(lon_A>0 and lon_B>0): #controlli per far che vada sempre a est 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf') #infinito
	if(lon_A<0 and lon_B<0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A>0 and lon_B<0):
		if((lon_A+abs(lon_B))<m.pi): #significa che sono nello stesso lato della terra.
			p=float('inf')  # esempio londra e dublin, deve fare tutto il giro.
		else:
			p=p
	if(lon_A<0 and lon_B>0):
		if((lon_B+abs(lon_A))<m.pi): 
			p=p
		else:
			p=float('inf')
	return(p*Rt) # trasforma l'arco in km.






dati=xlrd.open_workbook('worldcities.xlsx')
foglio = dati.sheet_by_name('Sheet1')

nR=foglio.nrows
nC=foglio.ncols


citta_inizio="London UK" #londra - nome
inizio="1826645935" # id - londra


# creiamo delle liste. 
nome=[]
lat=[]
log=[]
stato=[]
stato2=[] #non serve
pop=[] # population
iden=[]

pesi=[] 
nodi=[] # contiene tutti nodi. contiene n volte l'oggetto nodi.


for i in range(1,nR):	 #nR numero righe del excel. n-1 per intestazione. per quello range parte da riga 1. 
	nome.append(foglio.cell(i,0).value)
	lat.append(foglio.cell(i,2).value)
	log.append(foglio.cell(i,3).value)
	stato.append(foglio.cell(i,6).value)
	stato2.append(foglio.cell(i,7).value)
	pop.append(foglio.cell(i,9).value)
	iden.append(str(int(foglio.cell(i,10).value))) #riempo ogni lista

print("Lettura dati da file terminata") #solo per il print.











for i in range(0,nR-1): #parte da 0, perchè è la lista
	if(pop[i]==''):
		pop[i]=0 #per riempire con 0 i population vuote, perchè senno erano stringhe. 

print("Controllo popolazioni terminato")







controllo=[] #controllo da eliminare
for i in range(0,nR-1):
    if(lat[i]>180 or log[i]>180):   #90? absolute values?
        print(i)
        controllo.append(i)

for i in controllo :
	print("rimosso per coordinate errate: ",iden[i])
	nome.pop(i)
	lat.pop(i)
	log.pop(i)
	stato.pop(i)
	stato2.pop(i)
	pop.pop(i)
	iden.pop(i)



elim=[] #controlla long e lat uguali. 
elim1=[]
#controlla che se hanno stessa lat e long non abbiano lo stesso nome. senno eliminare. 
for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(lat[j]==lat[i] and log[j]==log[i] and iden[j]!=iden[i]):
			print(j)
			elim.append(j)

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(iden[j]==iden[i] and lat[j]!=lat[i] and log[j]!=log[i]): #per controllare se ci sono 2 città con le stesse coordinate ma diverso nome
			print(k)
			elim1.appen(j)

for j in elim:
	print("rimosso per coordinate coincidenti: ",iden[j])
	nome.pop(j) #pop elimina da ogni lista i doppioni.
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)

for j in elim1:
	print("rimosso per identificativo coincidente: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)


    
    
    
    
    
    
print(len(nome)) #aggiorna la quantità delle città dopo aver eliminato con i pop i doppioni. 
nR=len(nome)

citta_fine=citta_inizio+"_fine" #per fare il giro al mondo e arriva al inizio.
fine="1111111111" #inventiamo un id per la città di arrivo.

#aggiorna liste con citta finale, aggiunge la citta fine
for i in range(0,nR):
	if(iden[i]==inizio): #cerca l'id della città iniziale usando l'id
		nome.append(citta_fine) #append la città finale
		lat.append(lat[i])
		log.append(log[i]-0.1) #per evitare errore, non puo avere le stesse coordinate. 
		stato.append(stato[i])
		stato2.append(stato2[i])
		pop.append(pop[i])
		iden.append(fine) # aggiorna 'id con quello inventato 11111

print("Numerosità città") #aggiorna numero totale, contando l'ultima appena aggiunta.
print(len(nome))
nN=len(nome)

# fine prima parte--------------------------------









# inizio CALCOLO DISTANZA tra ogni città. 

for i in range(0,nN): # indice per ogni citta. voglio calcolare le 3 piu vicine.
	vicini=[] # crea lista di liste per ogni citta.
	for j in range(0,nN): # j parte da i, cerca le piu vicine. controlla tutte.
		if(iden[i]==inizio and iden[j] == fine): # da controllo, eliminare. 
			continue
		if(iden[j]==inizio):  # da controllo, eliminare. 
			continue
            
		if(iden[i]!=iden[j]): # se è la stessa città, niente, senno calcola la distanza
			d=dist2punti(lat[i],log[i],lat[j],log[j])
			if(len(vicini)==0): # se la lista è vuota, inserisco la prima citta e la sua info.
				vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]]) 
                #aggiunge una lista a la lista vicini
			else:	 # se la lista vicini non è vuota:
				for k in range(0,len(vicini)): # controllo i parametri dentro della lista e la distanza appena calcolata.
					if(d<vicini[k][1]): 
                # confronto onguna delle citta e comparo la distanza con d, se d è minore -> la inserisce in quella posizione. 
						vicini.insert(k,[iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
						if(len(vicini)>3):					
							vicini.pop(3) # se la lista vicini ha piu di 3 elementi, elimina il 4to. 
						break
					if(k==2 and len(vicini)<3): 
                    # voglio eliminare l'elemento con la d  maggiore. inserirli in ordine in vicini. 
						vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])

	diz={} #dentro dello stesso for, non cancello la lista vicini e faccio dizionario 
	for l in range(0,len(vicini)): # calcolo il peso di ogni citta - 3 vicine

		pes=2**(l+1) #fa la potenza. 2 alla 1, alla 2, alla 3. assegna il peso in base al ordine nella lista vicini. 
		if(vicini[l][2]>200000): #vicini, e la lista di liste, dentro di ogni lista, controllo population. 
			pes=pes+2

		if(vicini[l][3] != vicini[l][4]): # controllo i due stati, cambio peso. 
			pes=pes+2		
		diz[vicini[l][0]]=pes 
        # crea chiave diz con id di J e valore peso
        #aggiungo peso al dizionario. il peso della prima città, prendo l'id di ogni citta, e aggiungo peso

	print("Ancora "+str(nN-i)+" citta' da controllare" )

	pesi.append(diz) # questa è la lista di dizionari, ogni dizionario contiene 3 keys (j) con i suoi pesi. 
    
  #def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia):
	nodi.append(nodo(i,nome[i],lat[i],log[i],stato[i]+"-"+stato2[i],iden[i],True,float('inf'),"",pesi[i])) 
    #lista con tutta l' info di ogni nodo. float è il potenziali, definitivo, accumulando quelle prima. si aggiorna quando il flag diventa false.

    
    
# secondo -------------------   
    
    

print("Algoritmo di calcolo del grafo utilizzando l'algoritmo Dijkstra")





for x in nodi: # inizia il primo nodo, se inizia dalla citta iniziale
	if(x.identificativo == inizio):
		x.defi=False # cambia a false
		x.pot=0 # non è inf, è 0 perche inizia da li
		x.coll=inizio
		i=x.ide  # ? creavariabile i, le assegna l'ide, che è l' i del for precedente (l'ordine nella lista che scorre) 
k=0  #contatore
while True:
	for x in nodi[i].adia.keys(): # ogni nodo, ogni i, ognuno dei j delle liste, ?
		for y in nodi:
			if (y.identificativo == x and y.defi ):
				somma=nodi[i].pot + nodi[i].adia[x] # somma del peso piu peso prima.
				if(y.pot>somma): # se è minore di inf, lo definisco. sostituisco inf con la somma. 
					y.pot=somma
					y.coll=nodi[i].identificativo # coll: collegamento, il nodo da dove vengo, aggiungo id. 
	m=float('inf') # crea variabile m : inf
	for z in nodi:
		if(z.defi and z.pot<m): # controllo tutti nodi, se nodo non è stato controllato e pot è inf, aggiorno. 
			m=z.pot
			n=z.ide # aggiorno l'indice. Mi sto spostando da un graffo al seguente. mi sposto da città
	
	nodi[n].defi=False # lo cambio a false
	i=n # aggiorno l'indice.
	print(nodi[i].nome) # stampo nome della citta per tracking. ogni spostamento, non definitivo.

	if(nodi[i].identificativo == fine ): # uscita.
		lung=nodi[i].pot # definisco la lunghezza, è il potenziale del ultimo nodo. tutto il percorso.
		break
	k=k+1 #quanti nodi ha girato. non definitivo.

    
    
    
    
    
    
percorso=[] # per id
percorso_nomi=[]  #per nomi del percorso minimo, calcolato a ritroso








s=fine # parti dalla fine, suo id.
while True:
	percorso.append(s) # aggiungi citta fine.
	for x in nodi: #controllo tutti nodi controllati, vado al id di S:fine. vado indietro.
		if(x.identificativo == s):
			s=x.coll # aggiorna s alla precedendente
	if(s==inizio):
		percorso.append(s) # se incontri l'inizio fine. percorso accumula i id
		break


for x in percorso:
	for y in nodi:
		if(y.identificativo==x):
			percorso_nomi.append(y.nome+" ("+y.stato+")") # crea lista con nomi e stato.


strada=""
for i in range(0,len(percorso_nomi)-1):
	strada=strada+percorso_nomi[len(percorso_nomi)-1-i]+" - " #per stamparlo a ritroso.
strada=strada+" "+percorso_nomi[0] #le agrega al final la primer ciudad.

print("-------------------Risultato----------------------------")
print("Percorso terminato in "+str(lung)+" ore o "+str(float(lung)/24.)+" giorni. (tempo minimo)" )
print("Tragitto: " + strada)
print("--------------------------------------------------------")

Lettura dati da file terminata
Controllo popolazioni terminato
3913
2942
3976
19828
1155
5153
183
1426
6543
10176
3837
5609
10784
10848
5232
11750
4146
22961
4951
5365
5400
5744
18158
18157
1961
6709
rimosso per coordinate coincidenti:  1484228590
rimosso per coordinate coincidenti:  1484980825
rimosso per coordinate coincidenti:  1418863956
rimosso per coordinate coincidenti:  1840142185
rimosso per coordinate coincidenti:  1484504936
rimosso per coordinate coincidenti:  1840000395
rimosso per coordinate coincidenti:  1484559591
rimosso per coordinate coincidenti:  1076190259
rimosso per coordinate coincidenti:  1516928100
rimosso per coordinate coincidenti:  1840016003
rimosso per coordinate coincidenti:  1032425368
rimosso per coordinate coincidenti:  1840018841
rimosso per coordinate coincidenti:  1356820046
rimosso per coordinate coincidenti:  1076277699
rimosso per coordinate coincidenti:  1724017694
rimosso per coordinate coincidenti:  1804965832
rimosso per coordinate coinciden

Ancora 26348 citta' da controllare
Ancora 26347 citta' da controllare
Ancora 26346 citta' da controllare
Ancora 26345 citta' da controllare
Ancora 26344 citta' da controllare
Ancora 26343 citta' da controllare
Ancora 26342 citta' da controllare
Ancora 26341 citta' da controllare
Ancora 26340 citta' da controllare
Ancora 26339 citta' da controllare
Ancora 26338 citta' da controllare
Ancora 26337 citta' da controllare
Ancora 26336 citta' da controllare
Ancora 26335 citta' da controllare
Ancora 26334 citta' da controllare
Ancora 26333 citta' da controllare
Ancora 26332 citta' da controllare
Ancora 26331 citta' da controllare
Ancora 26330 citta' da controllare
Ancora 26329 citta' da controllare
Ancora 26328 citta' da controllare
Ancora 26327 citta' da controllare
Ancora 26326 citta' da controllare
Ancora 26325 citta' da controllare
Ancora 26324 citta' da controllare
Ancora 26323 citta' da controllare
Ancora 26322 citta' da controllare
Ancora 26321 citta' da controllare
Ancora 26320 citta' 

Ancora 26112 citta' da controllare
Ancora 26111 citta' da controllare
Ancora 26110 citta' da controllare
Ancora 26109 citta' da controllare
Ancora 26108 citta' da controllare
Ancora 26107 citta' da controllare
Ancora 26106 citta' da controllare
Ancora 26105 citta' da controllare
Ancora 26104 citta' da controllare
Ancora 26103 citta' da controllare
Ancora 26102 citta' da controllare
Ancora 26101 citta' da controllare
Ancora 26100 citta' da controllare
Ancora 26099 citta' da controllare
Ancora 26098 citta' da controllare
Ancora 26097 citta' da controllare
Ancora 26096 citta' da controllare
Ancora 26095 citta' da controllare
Ancora 26094 citta' da controllare
Ancora 26093 citta' da controllare
Ancora 26092 citta' da controllare
Ancora 26091 citta' da controllare
Ancora 26090 citta' da controllare
Ancora 26089 citta' da controllare
Ancora 26088 citta' da controllare
Ancora 26087 citta' da controllare
Ancora 26086 citta' da controllare
Ancora 26085 citta' da controllare
Ancora 26084 citta' 

Ancora 25644 citta' da controllare
Ancora 25643 citta' da controllare
Ancora 25642 citta' da controllare
Ancora 25641 citta' da controllare
Ancora 25640 citta' da controllare
Ancora 25639 citta' da controllare
Ancora 25638 citta' da controllare
Ancora 25637 citta' da controllare
Ancora 25636 citta' da controllare
Ancora 25635 citta' da controllare
Ancora 25634 citta' da controllare
Ancora 25633 citta' da controllare
Ancora 25632 citta' da controllare
Ancora 25631 citta' da controllare
Ancora 25630 citta' da controllare
Ancora 25629 citta' da controllare
Ancora 25628 citta' da controllare
Ancora 25627 citta' da controllare
Ancora 25626 citta' da controllare
Ancora 25625 citta' da controllare
Ancora 25624 citta' da controllare
Ancora 25623 citta' da controllare
Ancora 25622 citta' da controllare
Ancora 25621 citta' da controllare
Ancora 25620 citta' da controllare
Ancora 25619 citta' da controllare
Ancora 25618 citta' da controllare
Ancora 25617 citta' da controllare
Ancora 25616 citta' 

Ancora 25409 citta' da controllare
Ancora 25408 citta' da controllare
Ancora 25407 citta' da controllare
Ancora 25406 citta' da controllare
Ancora 25405 citta' da controllare
Ancora 25404 citta' da controllare
Ancora 25403 citta' da controllare
Ancora 25402 citta' da controllare
Ancora 25401 citta' da controllare
Ancora 25400 citta' da controllare
Ancora 25399 citta' da controllare
Ancora 25398 citta' da controllare
Ancora 25397 citta' da controllare
Ancora 25396 citta' da controllare
Ancora 25395 citta' da controllare
Ancora 25394 citta' da controllare
Ancora 25393 citta' da controllare
Ancora 25392 citta' da controllare
Ancora 25391 citta' da controllare
Ancora 25390 citta' da controllare
Ancora 25389 citta' da controllare
Ancora 25388 citta' da controllare
Ancora 25387 citta' da controllare
Ancora 25386 citta' da controllare
Ancora 25385 citta' da controllare
Ancora 25384 citta' da controllare
Ancora 25383 citta' da controllare
Ancora 25382 citta' da controllare
Ancora 25381 citta' 

...

Guatapará
Silveiras
Sales
Santa Bárbara do Rio Pardo
Diadema
São Vicente
Córdoba
Carpina
Linhares
Timbaúba
Estância
Palmeira dos Índios
Barreiros
Zinacantán
Escárcega
Miracema
Amatlán de los Reyes
Lake Elsinore
Porto Feliz
Tietê
Zacatelco
Socorro
Pirapora do Bom Jesus
Conchas
Fartura
Serra Azul
Piratininga
Mineiros do Tietê
Canyon Lake
São Bento do Sapucaí
Reginópolis
Iaras
Irapuã
Nuevo
Avaí
Maceió
João Pessoa
Feira de Santana
Taboão da Serra
Santo Agostinho
Itu
San Cristóbal
Cubatão
Santana de Parnaíba
Resende
Goiana
Champotón
Ocatlán
Frontera Corozal
Menifee
Campos do Jordão
Novo Horizonte
Wildomar
Barra Bonita
Dois Córregos
Cunha
Elias Fausto
Luís Antônio
Taguaí
Iacanga
Bananal
Rafard
Homeland
Pedra Bela
São Bernardo do Campo
Aracaju
Santos
Bauru
Cotia
Moreno Valley
Hemet
Pindamonhangaba
Jandira
Salto
Tlaxcala
Lerma
Taquaritinga
Monte Mor
Capivari
San Jacinto
Agudos
Santa Rita do Passa Quatro
Santa Rosa de Viterbo
Tambaú
Taquarituba
Águas de Lindóia
Borborema
Joanópolis
Torrinha
Are

Tibati
Gaoua
Djibo
Owerri
Ngaoundéré
Kaya
Bol
Gaoua
Guider
Mbé
Douala
Maiduguri
Guéckédou
Wa
Mao
Meïganga
Kombissiri
Léo
Boulsa
Abong Mbang
Gorom-Gorom
Sapouy
Aba
Umuahia
Bertoua
Edéa
Bama
Dori
Manga
Robertsport
Bopolu
Ikot Abasi
Zorgo
Bélabo
Maroua
Buea
Pala
Massakory
Batouri
Koupéla
Kalfou
Bogandé
Baboua
Oyem
Bouar
Tenkodogo
Kakata
Moussoro
Tubmanburg
Bogo
Pala Oua
Eséka
Pô
Ébolowa
Berbérati
Gbarnga
Fianga
N’Djamena
Monrovia
Bongor
Bozoum
Makokou
Massenya
Yao
Kribi
Upper Buchanan
Carnot
Malabo
Ati
Bensonville
Buchanan
Kelo
Mékambo
Bossangoa
Ebebiyín
Nola
Laï
Mongo
Cestos City
Oum Hadjer
Mikomeseng
Sémbé
Bossembele
Okondja
Limbe
Moundou
Bimbo
Doba
Mbaïki
Bitam
Kabo
Boloso
Abéché
Koumra
Ouésso
Ewo
Mongomo
Añisoc
Am-Timan
Kaga Bandoro
Biltine
Libenge
Bengonbeyene
Makoua
Mitzic
Bangui
Gemena
Sarh
Owando
Sibut
Zongo
Aconibe
El Geneina
Goz-Beida
Bambari
Impfondo
Koulamoutou
Ndélé
Birao
Bolobo
Bosobolo
Kouango
Bria
Zalingei
Binga
Mushie
Gamboma
Mbandaka
Gbadolite
Amdjarass
Basankusu
Moanda


Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt


Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt
Pālghāt


KeyboardInterrupt: 